In [20]:
import os
from PIL import Image
import numpy as np
from skimage.metrics import structural_similarity as ssim


In [2]:
def load_image_as_np(image_path:str):
    return np.asarray(Image.open(image_path).convert('RGB'), dtype=np.uint8)

def store_image_from_np(image_path:str,data:np.ndarray,format='RGB'):
    img = Image.fromarray(data, format)
    img.save(image_path)
    return img

In [21]:

def get_ssim(original_image,retrieved_image):
    return ssim(original_image,retrieved_image,channel_axis=2)
    
def get_mse(original_image,retrieved_image):
    #Mean Square Error
    diff = original_image-retrieved_image
    return (np.square(diff)).mean(axis=None)

def get_psnr(original_image,retrieved_image):
    #Peak signal to noise ratio
    mse = get_mse(original_image,retrieved_image)
    max = 255
    return 20*np.log10(max)-10*np.log10(mse)

metrics = {'mse':get_mse,
           'psnr':get_psnr,
           'ssim':get_ssim
}

In [45]:
wd_path = os.path.abspath(os.getcwd())
images_root_path = os.path.join(wd_path, "images","testing_jpeg")
images_list_file_path = os.path.join(images_root_path,"image_path_list.txt")

image_subdirs = ['92','90','88','85']
with open(images_list_file_path, "r") as image_list_file:
    image_path_list = image_list_file.readlines()



In [31]:
originals = {}
for image_name in image_path_list:
    originals[image_name.strip()] = load_image_as_np(os.path.join(images_root_path,'100_png',image_name.strip()))

In [35]:
def get_compression_stats(original_image,level_image):
    res = []
    for metric_key in metrics:
        res.append(metrics[metric_key](original_image,level_image))
    return res 

In [46]:
all_stats = {}
for level in image_subdirs:
    level_subdir = os.path.join(images_root_path,level)
    level_stats = []
    for original_image_name in originals.keys():
        image_level_path = os.path.join(level_subdir,original_image_name.replace('png','jpg'))
        image_level = load_image_as_np(image_level_path)
        image_original = originals[original_image_name]
        image_stats = get_compression_stats(image_original,image_level)
        level_stats.append(image_stats)
    level_stats = np.asarray(level_stats).mean(axis=0)
    print(level_stats)
    all_stats[level] = level_stats
    


[19.78542534 36.91955061  0.92901991]
[16.70587441 38.09260514  0.94084811]
[14.20856891 40.47007294  0.95033566]
[12.25508038 41.84034368  0.95768046]
